# Resume Generator

This interactive tool helps you create a professionally formatted resume from a template, customized with your information and optimized for job applications.

## Instructions:

1. Complete the form below:
   - Enter your full name (first and last)
   - Select a resume template from the dropdown
   - Choose whether to upload the final PDF to Google Drive

2. Click the "Generate Resume" button to create your resume.

3. Your files will be saved in the `data/output` directory.


In [ ]:
import os
import sys
import pickle
import tqdm.notebook as tqdm
import glob
import ipywidgets as widgets
from IPython.display import clear_output, display
from pathlib import Path

# Add scripts directory to path
sys.path.append('scripts')

# Import required modules
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from scripts.resume_generator import process_keywords, generate_resume, generate_pdf, upload_to_drive

# Get all available templates
template_files = glob.glob("templates/*.md")
template_names = [os.path.basename(f) for f in template_files]

# Create widgets for user input
name_input = widgets.Text(
    value='Malachi Dunn',
    description='Full Name:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

template_dropdown = widgets.Dropdown(
    options=template_names,
    value=template_names[0] if template_names else None,
    description='Select Template:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='70%')
)

upload_dropdown = widgets.Dropdown(
    options=[('Yes', True), ('No', False), ('Skip PDF generation', 'skip')],
    value=True,
    description='Upload to Google Drive:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

input_file_widget = widgets.Text(
    value='data/input/lead_gen.csv',
    description='Input Job Listings CSV:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='70%')
)

# Static config values
config_file = "templates/resume_config.yaml"
output_dir = "data/output"
drive_folder_id = "1s6n9eS9d2NNy9lyXgoYdPGOwkyGGujnw"

# Create a button to start the process
run_button = widgets.Button(
    description='Generate Resume',
    button_style='success',
    icon='rocket'
)

output_area = widgets.Output()

def on_run_button_clicked(b):
    with output_area:
        clear_output()
        
        # Get values from widgets
        basename = name_input.value.strip()
        if not basename:
            print("❌ Error: Please enter a name.")
            return
            
        template_name = template_dropdown.value
        if not template_name:
            print("❌ Error: Please select a template.")
            return
            
        template_file = os.path.join("templates", template_name)
        do_upload = upload_dropdown.value
        input_file = input_file_widget.value
        
        # Validate input file
        if not os.path.exists(input_file):
            print(f"❌ Error: Input file '{input_file}' not found.")
            return
        
        # Setup progress bar
        total_steps = 2
        if do_upload != 'skip':
            total_steps += 1
        
        progress = tqdm.tqdm(total=total_steps, desc="Resume Generator")
        
        # Create output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)
        
        # Process keywords
        keywords_output = os.path.join(output_dir, 'processed_keywords.csv')
        keywords = process_keywords(input_file, keywords_output)
        progress.update(1)
        
        # Generate resume
        basename_underscore = basename.replace(' ', '_').lower()
        base_output = os.path.join(output_dir, f"{basename_underscore}_resume.md")
        generate_resume(template_file, config_file, base_output, keywords_file=keywords_output)
        progress.update(1)
        
        # Generate PDF and upload to Google Drive
        if do_upload != 'skip':
            pdf_file = generate_pdf(base_output, output_dir)
            if pdf_file:
                if do_upload is True:
                    drive_link = upload_to_drive(pdf_file, drive_folder_id)
                    upload_message = f"and uploaded to Google Drive"
                else:
                    upload_message = f"(not uploaded to Google Drive)"
                progress.update(1)
            else:
                upload_message = f"but PDF generation failed"
        else:
            upload_message = "(PDF generation skipped)"
        
        progress.close()
        print(f"✅ Done! Your resume has been generated {upload_message}.")
        print(f"📄 Output files are in: {output_dir}")

# Connect the button to the function
run_button.on_click(on_run_button_clicked)

# Display the widgets
display(widgets.HTML("<h3>Resume Generator Options</h3>"))
display(name_input)
display(template_dropdown)
display(input_file_widget)
display(upload_dropdown)
display(run_button)
display(output_area)